In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# train_df = pd.read_csv("train.csv")

In [ ]:
# train_df.head()

In [ ]:
question = pd.read_csv("questions.csv") ### 13523 observations and 5 cols

In [ ]:
question.head()

In [ ]:
question.shape

In [ ]:
lecture = pd.read_csv('lectures.csv') ##418 lectures and 4 cols

In [ ]:
lecture.head()

In [ ]:
lecture.shape

# separate the tags col

In [ ]:
tag1 = question.tags.str.split(pat=" ", expand=True)
tag1 = tag1[~pd.isnull(tag1)] ##remove missing 

In [ ]:
tag_value = tag1[[0, 1, 2, 3]].values.ravel() ##convert several columns into array

In [ ]:
tag_unique = pd.unique(tag_value) ##get the unique values of tag

In [ ]:
tag_unique = tag_unique[~pd.isnull(tag_unique)] ##exclude missing

In [ ]:
len(tag_unique) 

In [ ]:
tag1 #why the expand make 6 cols with the max # tags is 4

There are 188 unique tags in the question dataset

In [ ]:
tag1.columns=['T1', 'T2', 'T3', 'T4', 'T5', 'T6']
tag1.head()

In [ ]:
tag2 = tag1.drop(columns=['T5', 'T6'])

In [ ]:
n_tag = tag2.count(axis=1)

In [ ]:
n_tag.describe()

In [ ]:
list_tag = tag2.values.tolist() ##covert the tag into list to explode

In [ ]:
len(list_tag)

In [ ]:
question_tag1 = pd.DataFrame(data = question['question_id'])

In [ ]:
question_tag1['tag'] = list_tag

In [ ]:
question_tag1.head()

In [ ]:
question_tag2 = question_tag1.explode('tag')

In [ ]:
question_tag2.head()

In [ ]:
question_tag3 = pd.get_dummies(question_tag2)

In [ ]:
question_tag = question_tag3.groupby(['question_id']).sum()

In [ ]:
question_tag['n_tags'] = question_tag.sum(axis=1)

In [ ]:
sum(question_tag['n_tags']-n_tag) ##make sure the data is converted right

In [ ]:
question.head()

In [ ]:
question_clean = question.merge(question_tag, left_on='question_id', right_on='question_id')

In [ ]:
question_clean.head()

In [ ]:
question_clean['bundle_count'] = question_clean.groupby(['bundle_id']).size()

In [ ]:
question_clean['bundle_binary'] = question_clean['bundle_count'].apply(lambda x: 0 if x <=1 else 1)

In [ ]:
question_clean.head()

In [ ]:
question_clean[['bundle_binary']].astype('category')

In [ ]:
question_clean['bundle_binary'].value_counts() ##there are 5372 questions are belong to bundles and 8151 are single questions

In [ ]:
question_clean['part_re'] = question_clean.apply(lambda x: 0 if x['part'] <=4 else 1, axis=1) ##binary; 0=listening, 1=reading

In [ ]:
question_clean.head()

# K-mean clustering

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale, StandardScaler
from sklearn.cluster import KMeans

In [ ]:
X = question_clean.iloc[:, 5:193]

In [ ]:
X.head()

In [ ]:
wcss = []
for i in range(1, 100):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(1, 100), wcss, 'bx-')
plt.title('Elbow plot for KMeans')
plt.xlabel('Number of clusters')
plt.ylabel('Within cluster sum-of-squares')

The pattern does not seem clear to me. May choose 40 clusters.

In [ ]:
kmeans = KMeans(n_clusters=40, random_state=0)
kmeans_pred_y = kmeans.fit_predict(X)

In [ ]:
kmeans_pred_y

In [ ]:
len(kmeans_pred_y)

In [ ]:
question_clean['kmean_cluster40'] = kmeans_pred_y

In [ ]:
question_clean.head()

In [ ]:
question_clean[['kmean_cluster40']].astype('category')

In [ ]:
question_clean['kmean_cluster'].value_counts()

# Hierarchical Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import adjusted_rand_score
import scipy.cluster.hierarchy as sch

In [ ]:
dendrogram = sch.dendrogram(sch.linkage(X, method = 'ward'))
plt.figure(figsize=(10, 3), dpi=300)
plt.title('Dendrogram')
plt.show()

Install networkX

In [ ]:
conda install -c anaconda networkx

In [ ]:
import networkx as nx

In [ ]:
edgelist = question_tag2.dropna()

In [ ]:
edgelist.head()

In [ ]:
edgelist.tag.astype(int)

In [ ]:
edgelist.to_csv('q_tag.txt', header=None, index=None)

In [ ]:
sn = nx.read_edgelist('q_tag.txt', create_using = nx.Graph(), delimiter=',', nodetype=int)

In [ ]:
print(nx.info(sn))

In [ ]:
nx.draw_networkx(sn)

In [ ]:
pos = nx.spring_layout(sn)
betCent = nx.betweenness_centrality(sn, normalized=True, endpoints=True)
node_color = [20000.0 * sn.degree(v) for v in sn]
node_size = [v * 10000 for v in betCent.values()]

In [ ]:
plt.figure(figsize=(20, 20), dpi=300)
nx.draw_networkx(sn, pos=pos, with_labels=False, node_color=node_color, node_size=node_size)
plt.axis('off')

In [ ]:
question_out = question_clean.iloc[:, [0, 193, 194, 196, 197]]

In [ ]:
question_out.head()

In [ ]:
question_out.to_csv('question_clean.csv')

# Lecture Data

In [ ]:
lecture.head()

In [ ]:
lecture.type_of.value_counts()

In [ ]:
lecture['part_re'] = lecture.apply(lambda x: 0 if x['part'] <= 4 else 1, axis=1)

In [ ]:
lecture[['part_re']].astype('category')

In [ ]:
lecture['part_re'].value_counts() #160 lectures are about listening and 258 are about reading

### Kmean for tags in lecture

In [ ]:
X = lecture[['tag']]

In [ ]:
wcss = []
for i in range(1, 20):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(range(1, 20), wcss, 'bx-')
plt.title('Elbow plot for KMeans')
plt.xlabel('Number of Clusters')
plt.ylabel('Within cluster sum-of-square')

The elbow plot shows there are five clusters

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(X)

In [ ]:
kmeans_pred_y = kmeans.predict(X)

In [ ]:
lecture['KMean_cluster5'] = kmeans_pred_y

In [ ]:
lecture.head()

In [ ]:
lecture.to_csv('lecture_clean.csv')